In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("../Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
#  YOUR CODE GOES HERE
application_df.drop(columns=['EIN', 'NAME'], axis=1, inplace=True)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
app_types_count = application_df.APPLICATION_TYPE.value_counts()

# Determine which values to replace if counts are less than ...?
#  YOUR CODE GOES HERE
replace_application = list(app_types_count[app_types_count < 500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [4]:
classification_count = application_df.CLASSIFICATION.value_counts()
replace_class = list(classification_count[classification_count <1800].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [5]:
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()

In [6]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
#  YOUR CODE GOES HERE
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [7]:
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df.drop(application_cat,axis= 1, inplace= True)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [8]:
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop("IS_SUCCESSFUL", axis= 1).values

# Split the preprocessed data into a training and testing dataset
#  YOUR CODE GOES HERE
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [9]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [10]:
#attempt 1, additional nodes in hidden layers
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 40

nn = tf.keras.models.Sequential()

# First hidden layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 80)                3520      
_________________________________________________________________
dense_1 (Dense)              (None, 40)                3240      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 41        
Total params: 6,801
Trainable params: 6,801
Non-trainable params: 0
_________________________________________________________________


In [11]:
# Compile the model
#  YOUR CODE GOES HERE
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

#checkpoint function to save weights every 5 epochs
import os
from tensorflow.keras.callbacks import ModelCheckpoint

os.makedirs("checkpoints_optimized_1/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=804*5)

In [12]:
fit_model = nn.fit(X_train,y_train,epochs=50,callbacks=[cp_callback])

Epoch 1/50
804/804 [==============================] - 1s 991us/step - loss: 46905.5469 - accuracy: 0.4962
Epoch 2/50
804/804 [==============================] - 1s 1ms/step - loss: 21913.1562 - accuracy: 0.4871
Epoch 3/50
804/804 [==============================] - 1s 908us/step - loss: 11062.5244 - accuracy: 0.5038
Epoch 4/50
804/804 [==============================] - 1s 851us/step - loss: 21835.1328 - accuracy: 0.4945
Epoch 5/50
758/804 [===========================>..] - ETA: 0s - loss: 9837.2529 - accuracy: 0.5094 
Epoch 00005: saving model to checkpoints\weights.05.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 9314.2783 - accuracy: 0.5084
Epoch 6/50
804/804 [==============================] - 1s 1ms/step - loss: 1978.3119 - accuracy: 0.5166
Epoch 7/50
804/804 [==============================] - 1s 1ms/step - loss: 7615.5464 - accuracy: 0.5039
Epoch 8/50
804/804 [==============================] - 1s 1ms/step - loss: 19924.5840 - accuracy: 0.5194
Epoch 9/50
804/804 

In [13]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 1.0868 - accuracy: 0.6442
Loss: 1.0868315696716309, Accuracy: 0.6441982388496399


In [14]:
nn.save("AlphabetSoupCharityOptimize1.h5")

In [15]:
#result of attempt 1: high accuracy, but high loss makes this model no good

In [16]:
#attempt 2, additional layers
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 40
hidden_nodes_layer3 = 20

nn_2 = tf.keras.models.Sequential()

# First hidden layer
#  YOUR CODE GOES HERE
nn_2.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
#  YOUR CODE GOES HERE
nn_2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn_2.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
#  YOUR CODE GOES HERE
nn_2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 80)                3520      
_________________________________________________________________
dense_4 (Dense)              (None, 40)                3240      
_________________________________________________________________
dense_5 (Dense)              (None, 20)                820       
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 21        
Total params: 7,601
Trainable params: 7,601
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Compile the model
#  YOUR CODE GOES HERE
nn_2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
#new folder for new model
os.makedirs("checkpoints_optimized_2/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [18]:
fit_model = nn_2.fit(X_train,y_train,epochs=50,callbacks=[cp_callback])

Epoch 1/50
804/804 [==============================] - 1s 1ms/step - loss: 30758.9512 - accuracy: 0.4921: 0s - loss: 204
Epoch 2/50
804/804 [==============================] - 1s 892us/step - loss: 15278.1680 - accuracy: 0.49890s - loss: 786.2438 - 
Epoch 3/50
804/804 [==============================] - 1s 944us/step - loss: 6239.4282 - accuracy: 0.5073 0s - loss: 6703.3921 - accuracy: 0.50
Epoch 4/50
804/804 [==============================] - 1s 925us/step - loss: 20609.2734 - accuracy: 0.49490s - loss: 30094.1016 - ac
Epoch 5/50
754/804 [===========================>..] - ETA: 0s - loss: 11853.8145 - accuracy: 0.4844 ETA: 0s - loss: 8927.4756 - accuracy: 0.516 - ETA: 0s - loss: 5356.0371 - accur - ETA: 0s - loss: 12797.4814 - accuracy: 0. - ETA: 0s - loss: 11292.6230 - accuracy: 0.4847
Epoch 00005: saving model to checkpoints\weights.05.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 10769.7363 - accuracy: 0.4832
Epoch 6/50
804/804 [==============================] - 1

804/804 [==============================] - 1s 1ms/step - loss: 0.6909 - accuracy: 0.5335: 0s - loss: 0.6918 - accuracy:  - ETA: 0s - loss: 0.6
Epoch 50/50
791/804 [============================>.] - ETA: 0s - loss: 0.6910 - accuracy: 0.5331
Epoch 00050: saving model to checkpoints\weights.50.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6909 - accuracy: 0.5335


In [19]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6532 - accuracy: 0.6367
Loss: 0.6531938910484314, Accuracy: 0.636734664440155


In [20]:
nn_2.save("AlphabetSoupCharityOptimize2.h5")

In [21]:
#result of attempt 2: higher accuracy than original, less than previous attempt, and lower loss than last attempt. pretty good!

In [22]:
#attempt 3, additional layers and activation function changed to sigmoid
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 40
hidden_nodes_layer3 = 20

nn_3 = tf.keras.models.Sequential()

# First hidden layer
#  YOUR CODE GOES HERE
nn_3.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="sigmoid"))

# Second hidden layer
#  YOUR CODE GOES HERE
nn_3.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

# Third hidden layer
nn_3.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# Output layer
#  YOUR CODE GOES HERE
nn_3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_3.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 80)                3520      
_________________________________________________________________
dense_8 (Dense)              (None, 40)                3240      
_________________________________________________________________
dense_9 (Dense)              (None, 20)                820       
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 21        
Total params: 7,601
Trainable params: 7,601
Non-trainable params: 0
_________________________________________________________________


In [23]:
# Compile the model
#  YOUR CODE GOES HERE
nn_3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
#new folder for new model
os.makedirs("checkpoints_optimized_3/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [24]:
fit_model = nn_3.fit(X_train,y_train,epochs=50,callbacks=[cp_callback])

Epoch 1/50
804/804 [==============================] - 1s 1ms/step - loss: 0.6928 - accuracy: 0.5287: 0s - loss: 0.6933 - accura
Epoch 2/50
804/804 [==============================] - 1s 1ms/step - loss: 0.6915 - accuracy: 0.5309: 0s - loss: 0.6912 - accuracy
Epoch 3/50
804/804 [==============================] - 1s 989us/step - loss: 0.6913 - accuracy: 0.5313
Epoch 4/50
804/804 [==============================] - ETA: 0s - loss: 0.6915 - accuracy: 0.5321 ETA: 0s - loss: 0.6901 - accu - ETA: 0s - loss: 0.6914  - 1s 997us/step - loss: 0.6914 - accuracy: 0.5328
Epoch 5/50
768/804 [===========================>..] - ETA: 0s - loss: 0.6914 - accuracy: 0.5318 ETA: 0s - loss: 0.690
Epoch 00005: saving model to checkpoints\weights.05.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6914 - accuracy: 0.5321
Epoch 6/50
804/804 [==============================] - 1s 1ms/step - loss: 0.6914 - accuracy: 0.5322: 0s - loss: 0.6905 - accu - ETA: 0s - loss: 0.6920 - accura - ETA: 0s - lo

804/804 [==============================] - 1s 976us/step - loss: 0.6913 - accuracy: 0.53350s - loss: 0.6920 -  - ETA: 0s - loss: 0.6912 - accuracy: 0.
Epoch 49/50
804/804 [==============================] - ETA: 0s - loss: 0.6912 - accuracy: 0.5334 ETA: 0s - loss: 0.6904  - 1s 1ms/step - loss: 0.6912 - accuracy: 0.5335
Epoch 50/50
790/804 [============================>.] - ETA: 0s - loss: 0.6912 - accuracy: 0.5330
Epoch 00050: saving model to checkpoints\weights.50.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5335


In [25]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6917 - accuracy: 0.5292
Loss: 0.6917336583137512, Accuracy: 0.5292128324508667


In [26]:
nn_3.save("AlphabetSoupCharityOptimize3.h5")

In [27]:
#result of attempt 3: no good, accuracy is down, and loss is up

In [28]:
#attempt 4: further bucketting
application_bucket_df = pd.read_csv("../Resources/charity_data.csv")
application_bucket_df.drop(columns=['EIN', 'NAME'], axis=1, inplace=True)
application_cat = application_bucket_df.dtypes[application_bucket_df.dtypes == "object"].index.tolist()
app_types_count = application_bucket_df.APPLICATION_TYPE.value_counts()

#modified buckets
replace_application = list(app_types_count[app_types_count < 1000].index)

# Replace in dataframe
for app in replace_application:
    application_bucket_df.APPLICATION_TYPE = application_bucket_df.APPLICATION_TYPE.replace(app,"Other")

classification_count = application_bucket_df.CLASSIFICATION.value_counts()

replace_class = list(classification_count[classification_count < 2000].index)

# Replace in dataframe
for cls in replace_class:
    application_bucket_df.CLASSIFICATION = application_bucket_df.CLASSIFICATION.replace(cls,"Other")

enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
#  YOUR CODE GOES HERE
encode_df = pd.DataFrame(enc.fit_transform(application_bucket_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)

application_bucket_df = application_bucket_df.merge(encode_df,left_index=True, right_index=True)
application_bucket_df.drop(application_cat,axis= 1, inplace= True)

y = application_bucket_df["IS_SUCCESSFUL"].values
X = application_bucket_df.drop("IS_SUCCESSFUL", axis= 1).values

# Split the preprocessed data into a training and testing dataset
#  YOUR CODE GOES HERE
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [29]:
#back to our most successful model
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 40
hidden_nodes_layer3 = 20

nn_4 = tf.keras.models.Sequential()

# First hidden layer
#  YOUR CODE GOES HERE
nn_4.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
#  YOUR CODE GOES HERE
nn_4.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn_4.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
#  YOUR CODE GOES HERE
nn_4.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_4.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 80)                3120      
_________________________________________________________________
dense_12 (Dense)             (None, 40)                3240      
_________________________________________________________________
dense_13 (Dense)             (None, 20)                820       
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 21        
Total params: 7,201
Trainable params: 7,201
Non-trainable params: 0
_________________________________________________________________


In [30]:
# Compile the model
#  YOUR CODE GOES HERE
nn_4.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
#new folder for new model
os.makedirs("checkpoints_optimized_4/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [31]:
fit_model = nn_4.fit(X_train,y_train,epochs=50,callbacks=[cp_callback])

Epoch 1/50
804/804 [==============================] - 1s 1ms/step - loss: 35745.4688 - accuracy: 0.4849
Epoch 2/50
804/804 [==============================] - 1s 1ms/step - loss: 14500.4160 - accuracy: 0.4834
Epoch 3/50
804/804 [==============================] - 1s 1ms/step - loss: 3470.4429 - accuracy: 0.4858
Epoch 4/50
804/804 [==============================] - 1s 1ms/step - loss: 7748.6514 - accuracy: 0.4916A: 0s - loss: 5016.5059 - accuracy: 0.480 - ETA: 0s - loss: 4878.7593 - accuracy: 0 - ETA: 0s - loss: 7636.0098 - accuracy: 0.48
Epoch 5/50
760/804 [===========================>..] - ETA: 0s - loss: 21742.7090 - accuracy: 0.4956 ETA: 0s - loss: 248.7977 - accuracy: 0.54 - ETA: 0s - loss: 4349.0713 - accu
Epoch 00005: saving model to checkpoints\weights.05.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 20760.1797 - accuracy: 0.4980
Epoch 6/50
804/804 [==============================] - 1s 1ms/step - loss: 5639.1885 - accuracy: 0.5017
Epoch 7/50
804/804 [========

768/804 [===========================>..] - ETA: 0s - loss: 0.6910 - accuracy: 0.5339 ETA: 0s - loss: - ETA: 0s - loss: 0.6909 - accuracy: 0.53 - ETA: 0s - loss: 0.6912 - accuracy: 0.5325
Epoch 00045: saving model to checkpoints\weights.45.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6910 - accuracy: 0.5335
Epoch 46/50
804/804 [==============================] - 1s 986us/step - loss: 0.6909 - accuracy: 0.5335
Epoch 47/50
804/804 [==============================] - 1s 984us/step - loss: 0.6909 - accuracy: 0.53350s - loss: 0.6922 - accu - ETA: 0s - loss: 0.6917 - ac
Epoch 48/50
804/804 [==============================] - 1s 994us/step - loss: 0.6910 - accuracy: 0.5335
Epoch 49/50
804/804 [==============================] - 1s 992us/step - loss: 0.6910 - accuracy: 0.53350s - loss: 0.6928 -  - ETA: 0s - loss: 0.6916 - ac
Epoch 50/50
772/804 [===========================>..] - ETA: 0s - loss: 0.6910 - accuracy: 0.5335 ETA: 0s - loss: 0.6865 - accura - ETA: 0s - loss: 0.69

In [32]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_4.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6838 - accuracy: 0.6385
Loss: 0.6837666034698486, Accuracy: 0.6384839415550232


In [33]:
nn_4.save("AlphabetSoupCharityOptimize4.h5")

In [ ]:
#results from attempt 4: slightly higher accuracy than attempt 2, but also greater loss